In [14]:
def create_car_graph(place="Montréal, Quebec, Canada"):
    """Create a driving network graph for Montreal"""
    G = ox.graph_from_place(place, network_type="drive", simplify=True)
    
    for n, data in G.nodes(data=True):
        if "x" in data and "y" in data:
            lon, lat = data["x"], data["y"]
        elif "geometry" in data:
            lon, lat = data["geometry"].x, data["geometry"].y
        else:
            lon, lat = None, None
        data["lon"] = lon
        data["lat"] = lat

    CAR_SPEED_M_S = 50.0 / 3.6  # ~50 km/h average city driving in m/s
    
    for u, v, key, data in G.edges(keys=True, data=True):
        length = data.get("length")
        if length is None:
            if "geometry" in data:
                length = data["geometry"].length
            else:
                length = 0.0
        data["length"] = length
        travel_time = length / CAR_SPEED_M_S
        data["weight"] = travel_time
        data["mode"] = "car"
        data["distance_m"] = length
    return G

def merge_car_and_subway(G_car, G_subway, access_threshold_m=300, walking_speed_m_s=WALKING_SPEED_M_S, parking_penalty_s=120):

    G_combined = nx.MultiDiGraph()

    for n, d in G_car.nodes(data=True):
        node_data = d.copy()
        node_data["type"] = "car_node"
        G_combined.add_node(n, **node_data)
        
    for n, d in G_subway.nodes(data=True):
        if G_combined.has_node(n):
            existing = G_combined.nodes[n]
            existing.update(d)
            existing["type"] = "station"
        else:
            node_data = d.copy()
            node_data["type"] = "station"
            G_combined.add_node(n, **node_data)
    for u, v, key, data in G_car.edges(keys=True, data=True):
        G_combined.add_edge(u, v, **data)
        
    for u, v, data in G_subway.edges(data=True):
        lines = list(data.get("lines", []))
        weight = data.get("weight")
        dist = data.get("distance_m")
        G_combined.add_edge(u, v, mode="subway", weight=weight, lines=lines, distance_m=dist)
        G_combined.add_edge(v, u, mode="subway", weight=weight, lines=lines, distance_m=dist)

    car_node_list = []
    car_coords = []
    for n, d in G_car.nodes(data=True):
        if d.get("lon") is None or d.get("lat") is None:
            continue
        x, y = project(d["lon"], d["lat"])
        car_node_list.append(n)
        car_coords.append((x, y))
    car_coords = np.array(car_coords)
    tree = cKDTree(car_coords)

    for station in G_subway.nodes():
        s_data = G_subway.nodes[station]
        if s_data.get("lon") is None or s_data.get("lat") is None:
            continue
        sx, sy = project(s_data["lon"], s_data["lat"])
        
        idxs = tree.query_ball_point((sx, sy), r=access_threshold_m)
        if not idxs:
            _, idx = tree.query((sx, sy))
            idxs = [idx]
            
        for idx in idxs:
            car_node = car_node_list[idx]
            cx, cy = car_coords[idx]
            dist = math.hypot(sx - cx, sy - cy)
            walk_time = dist / walking_speed_m_s
            total_cost = walk_time + parking_penalty_s
            G_combined.add_edge(
                car_node, station, 
                mode="park_and_ride", 
                weight=total_cost, 
                distance_m=dist, 
                description="park car and walk to subway"
            )
            
    return G_combined

In [6]:
import os
import json
import zipfile
import io
import datetime

import requests
import pandas as pd
import networkx as nx
import numpy as np
from scipy.spatial import cKDTree
from pyproj import Transformer
from shapely.geometry import Point, LineString
import geopandas as gpd
import matplotlib.pyplot as plt
import osmnx as ox
from networkx.readwrite import json_graph
import math

WALKING_SPEED_M_S = 1.4           # ~5 km/h
BIKE_SPEED_M_S = 5.0              # ~18 km/h
SUBWAY_SPEED_KMH = 30.0           # approx average
SUBWAY_SPEED_M_S = SUBWAY_SPEED_KMH * 1000 / 3600

ACCESS_THRESHOLD_M = 150          # subway <-> walk / bike transfer radius
BIKE_ACCESS_THRESHOLD_M = 200     # for bike<->subway
BIXI_SNAP_RADIUS_M = 100          # snapping BIXI to bike network
DOCKING_PENALTY_S = 20            # bike -> BIXI docking overhead
MOUNT_PENALTY_S = 10              # BIXI -> bike mounting overhead
STATION_WALK_TRANSFER_PENALTY_S = 10  # subway <-> BIXI walking transfer extra overhead

BIXI_PREFIX = "bixi_"


_transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
def project(lon, lat):
    return _transformer.transform(lon, lat)  # x, y in meters


In [7]:
def download_and_extract_gtfs(gtfs_url="http://www.stm.info/sites/default/files/gtfs/gtfs_stm.zip"):
    resp = requests.get(gtfs_url, timeout=30)
    resp.raise_for_status()
    z = zipfile.ZipFile(io.BytesIO(resp.content))

    def read_csv(name):
        with z.open(name) as f:
            return pd.read_csv(f, dtype=str)

    routes = read_csv("routes.txt")
    trips = read_csv("trips.txt")
    stop_times = read_csv("stop_times.txt")
    stops = read_csv("stops.txt")
    return routes, trips, stop_times, stops

def build_subway_graph_from_gtfs(routes, trips, stop_times, stops):
    # subway / metro route_type == "1"
    metro_routes = routes[routes["route_type"] == "1"].copy()
    if metro_routes.empty:
        raise RuntimeError("No metro routes in GTFS")

    metro_trips = trips[trips["route_id"].isin(metro_routes["route_id"])]
    representative = (
        metro_trips.sort_values(["route_id", "direction_id", "trip_id"])
        .drop_duplicates(subset=["route_id", "direction_id"], keep="first")
    )
    chosen_trip_ids = set(representative["trip_id"])

    relevant_stop_times = stop_times[stop_times["trip_id"].isin(chosen_trip_ids)].copy()
    relevant_stop_times["stop_sequence"] = relevant_stop_times["stop_sequence"].astype(int)
    relevant_stop_times.sort_values(["trip_id", "stop_sequence"], inplace=True)

    stops_idx = stops.set_index("stop_id")

    G_subway = nx.Graph()
    used_stop_ids = set(relevant_stop_times["stop_id"])
    for sid in used_stop_ids:
        if sid not in stops_idx.index:
            continue
        row = stops_idx.loc[sid]
        lat = float(row["stop_lat"])
        lon = float(row["stop_lon"])
        name = row.get("stop_name", sid)
        G_subway.add_node(sid, name=name, lat=lat, lon=lon, lines=set(), type="station")

    trip_groups = relevant_stop_times.groupby("trip_id")
    for trip_id, group in trip_groups:
        seq = list(group["stop_id"])
        route_id = representative.loc[representative["trip_id"] == trip_id, "route_id"].iloc[0]
        route_row = routes[routes["route_id"] == route_id].iloc[0]
        line_name = route_row.get("route_long_name") or route_row.get("route_short_name") or route_id
        for a, b in zip(seq, seq[1:]):
            if not G_subway.has_node(a) or not G_subway.has_node(b):
                continue
            G_subway.nodes[a]["lines"].add(line_name)
            G_subway.nodes[b]["lines"].add(line_name)
            if G_subway.has_edge(a, b):
                G_subway.edges[a, b]["lines"].add(line_name)
            else:
                G_subway.add_edge(a, b, lines={line_name})

    for u, v, data in G_subway.edges(data=True):
        if "weight" in data:
            continue
        u_lon, u_lat = G_subway.nodes[u]["lon"], G_subway.nodes[u]["lat"]
        v_lon, v_lat = G_subway.nodes[v]["lon"], G_subway.nodes[v]["lat"]
        ux, uy = project(u_lon, u_lat)
        vx, vy = project(v_lon, v_lat)
        dist = math.hypot(ux - vx, uy - vy)  # meters
        travel_time = dist / SUBWAY_SPEED_M_S + 15  # dwell included
        G_subway.edges[u, v]["weight"] = travel_time
        G_subway.edges[u, v]["mode"] = "subway"
        G_subway.edges[u, v]["distance_m"] = dist
    return G_subway


In [ ]:
def create_walk_graph(place="Montréal, Quebec, Canada"):
    G = ox.graph_from_place(place, network_type="walk", simplify=True)
    
    for n, data in G.nodes(data=True):
        if "x" in data and "y" in data:
            lon, lat = data["x"], data["y"]
        elif "geometry" in data:
            lon, lat = data["geometry"].x, data["geometry"].y
        else:
            lon, lat = None, None
        data["lon"] = lon
        data["lat"] = lat

    for u, v, key, data in G.edges(keys=True, data=True):
        length = data.get("length")
        if length is None:
            if "geometry" in data:
                length = data["geometry"].length
            else:
                length = 0.0
        data["length"] = length
        travel_time = length / WALKING_SPEED_M_S
        data["weight"] = travel_time
        data["travel_time"] = travel_time  # Ensure both fields for compatibility
        data["mode"] = "walk"
        data["distance_m"] = length
    return G

def create_bike_graph(place="Montréal, Quebec, Canada"):
    G = ox.graph_from_place(place, network_type="bike", simplify=True)
    for n, data in G.nodes(data=True):
        if "x" in data and "y" in data:
            lon, lat = data["x"], data["y"]
        elif "geometry" in data:
            lon, lat = data["geometry"].x, data["geometry"].y
        else:
            lon, lat = None, None
        data["lon"] = lon
        data["lat"] = lat

    for u, v, key, data in G.edges(keys=True, data=True):
        length = data.get("length")
        if length is None:
            if "geometry" in data:
                length = data["geometry"].length
            else:
                length = 0.0
        data["length"] = length
        travel_time = length / BIKE_SPEED_M_S
        data["weight"] = travel_time
        data["travel_time"] = travel_time  # Ensure both fields for compatibility
        data["mode"] = "bike"
        data["distance_m"] = length
    return G


In [9]:
def merge_walk_and_subway(G_walk, G_subway, access_threshold_m=ACCESS_THRESHOLD_M, walking_speed_m_s=WALKING_SPEED_M_S, access_penalty_s=30):
    G_combined = nx.MultiGraph()

    # Add nodes
    for n, d in G_walk.nodes(data=True):
        node_data = d.copy()
        node_data["type"] = "walk_node"
        G_combined.add_node(n, **node_data)
        
    for n, d in G_subway.nodes(data=True):
        if G_combined.has_node(n):
            existing = G_combined.nodes[n]
            existing.update(d)
            existing["type"] = "station"
        else:
            node_data = d.copy()
            node_data["type"] = "station"
            G_combined.add_node(n, **node_data)

    for u, v, key, data in G_walk.edges(keys=True, data=True):
        G_combined.add_edge(u, v, **data)
    for u, v, data in G_subway.edges(data=True):
        edge_data = data.copy()
        edge_data["lines"] = list(edge_data.get("lines", []))
        G_combined.add_edge(u, v, **edge_data)

    walk_node_list = []
    walk_coords = []
    for n, d in G_walk.nodes(data=True):
        if d.get("lon") is None or d.get("lat") is None:
            continue
        x, y = project(d["lon"], d["lat"])
        walk_node_list.append(n)
        walk_coords.append((x, y))
    walk_coords = np.array(walk_coords)
    tree = cKDTree(walk_coords)

    for station in G_subway.nodes():
        s_data = G_subway.nodes[station]
        if s_data.get("lon") is None or s_data.get("lat") is None:
            continue
        sx, sy = project(s_data["lon"], s_data["lat"])
        idxs = tree.query_ball_point((sx, sy), r=access_threshold_m)
        if not idxs:
            _, idx = tree.query((sx, sy))
            idxs = [idx]
        for idx in idxs:
            walk_node = walk_node_list[idx]
            wx, wy = walk_coords[idx]
            dist = math.hypot(sx - wx, sy - wy)
            walk_time = dist / walking_speed_m_s
            total_cost = walk_time + access_penalty_s
            # bidirectional
            G_combined.add_edge(station, walk_node, mode="walk_access", weight=total_cost, distance_m=dist, description="subway<->walk transfer")
            G_combined.add_edge(walk_node, station, mode="walk_access", weight=total_cost, distance_m=dist, description="walk<->subway transfer")
    return G_combined

def merge_bike_and_subway(G_bike, G_subway, access_threshold_m=BIKE_ACCESS_THRESHOLD_M, walking_speed_m_s=WALKING_SPEED_M_S, access_penalty_s=30):
    G_combined = nx.MultiDiGraph()

    for n, d in G_bike.nodes(data=True):
        node_data = d.copy()
        node_data["type"] = "bike_node"
        G_combined.add_node(n, **node_data)
        
    for n, d in G_subway.nodes(data=True):
        if G_combined.has_node(n):
            existing = G_combined.nodes[n]
            existing.update(d)
            existing["type"] = "station"
        else:
            node_data = d.copy()
            node_data["type"] = "station"
            G_combined.add_node(n, **node_data)

    for u, v, key, data in G_bike.edges(keys=True, data=True):
        G_combined.add_edge(u, v, **data)
    for u, v, data in G_subway.edges(data=True):
        lines = list(data.get("lines", []))
        weight = data.get("weight")
        dist = data.get("distance_m")
        G_combined.add_edge(u, v, mode="subway", weight=weight, lines=lines, distance_m=dist)
        G_combined.add_edge(v, u, mode="subway", weight=weight, lines=lines, distance_m=dist)

    bike_node_list = []
    bike_coords = []
    for n, d in G_bike.nodes(data=True):
        if d.get("lon") is None or d.get("lat") is None:
            continue
        x, y = project(d["lon"], d["lat"])
        bike_node_list.append(n)
        bike_coords.append((x, y))
    bike_coords = np.array(bike_coords)
    tree = cKDTree(bike_coords)

    for station in G_subway.nodes():
        s_data = G_subway.nodes[station]
        if s_data.get("lon") is None or s_data.get("lat") is None:
            continue
        sx, sy = project(s_data["lon"], s_data["lat"])
        idxs = tree.query_ball_point((sx, sy), r=access_threshold_m)
        if not idxs:
            _, idx = tree.query((sx, sy))
            idxs = [idx]
        for idx in idxs:
            bike_node = bike_node_list[idx]
            bx, by = bike_coords[idx]
            dist = math.hypot(sx - bx, sy - by)
            walk_time = dist / walking_speed_m_s
            total_cost = walk_time + access_penalty_s
            G_combined.add_edge(station, bike_node, mode="bike_access", weight=total_cost, distance_m=dist, description="subway<->bike transfer")
            G_combined.add_edge(bike_node, station, mode="bike_access", weight=total_cost, distance_m=dist, description="bike<->subway transfer")
    return G_combined

In [10]:
def fetch_bixi_gbfs(root_urls=None):
    if root_urls is None:
        root_urls = [
            "https://api-core.bixi.com/gbfs/gbfs.json",
            "https://gbfs.velobixi.com/gbfs/gbfs.json",
        ]
    last_exc = None
    for root in root_urls:
        try:
            r = requests.get(root, timeout=10)
            r.raise_for_status()
            feed_index = r.json()
            data = feed_index.get("data", {})
            if "en" in data:
                feeds = data["en"]["feeds"]
            else:
                first = next(iter(data.values()))
                feeds = first["feeds"]
            info_url = None
            status_url = None
            for f in feeds:
                name = f.get("name", "")
                if "station_information" in name:
                    info_url = f["url"]
                elif "station_status" in name:
                    status_url = f["url"]
            if not info_url:
                continue
                
            stations_resp = requests.get(info_url, timeout=10)
            stations_resp.raise_for_status()
            stations_json = stations_resp.json()
            
            stations_data = stations_json["data"]["stations"]
            stations_df = pd.DataFrame(stations_data)
            
            status_df = None
            if status_url:
                try:
                    status_resp = requests.get(status_url, timeout=10)
                    status_resp.raise_for_status()
                    status_json = status_resp.json()
                    status_data = status_json["data"]["stations"]
                    status_df = pd.DataFrame(status_data)
                except Exception as e:
                    print(f"Warning: Could not fetch status data: {e}")
                    
            return stations_df, status_df
        except Exception as e:
            last_exc = e
            continue
    raise RuntimeError(f"Failed to fetch BIXI GBFS: {last_exc}")

def integrate_bixi_as_endpoints(G_bike, stations_df, status_df=None, snap_radius_m=BIXI_SNAP_RADIUS_M):
    bike_node_list = []
    bike_coords = []
    for n, d in G_bike.nodes(data=True):
        if d.get("lon") is None or d.get("lat") is None:
            continue
        x, y = project(d["lon"], d["lat"])
        bike_node_list.append(n)
        bike_coords.append((x, y))
    if not bike_coords:
        raise RuntimeError("No coordinate-tagged bike nodes")
    bike_coords = np.array(bike_coords)
    tree = cKDTree(bike_coords)

    G = nx.MultiDiGraph()
    for n, d in G_bike.nodes(data=True):
        G.add_node(n, **d, type=d.get("type", "bike_node"))
    for u, v, key, data in G_bike.edges(keys=True, data=True):
        G.add_edge(u, v, **data)

    for _, row in stations_df.iterrows():
        station_id = str(row["station_id"])
        station_node = f"{BIXI_PREFIX}{station_id}"
        lat = float(row["lat"])
        lon = float(row["lon"])
        name = row.get("name") or row.get("station_name") or station_id
        capacity = row.get("capacity", None)
        G.add_node(
            station_node,
            name=name,
            lat=lat,
            lon=lon,
            type="bixi_station",
            capacity=capacity,
        )
        sx, sy = project(lon, lat)
        idxs = tree.query_ball_point((sx, sy), r=snap_radius_m)
        if not idxs:
            _, idx = tree.query((sx, sy))
            idxs = [idx]
        for idx in idxs:
            bike_node = bike_node_list[idx]
            bx, by = bike_coords[idx]
            dist = math.hypot(sx - bx, sy - by)
            ride_time = dist / BIKE_SPEED_M_S
            # bike -> BIXI (dock)
            G.add_edge(
                bike_node,
                station_node,
                mode="bike_to_bixi",
                weight=ride_time + DOCKING_PENALTY_S,
                distance_m=dist,
                description="ride to BIXI and dock"
            )
            # BIXI -> bike (mount)
            G.add_edge(
                station_node,
                bike_node,
                mode="bixi_to_bike",
                weight=MOUNT_PENALTY_S,
                distance_m=dist,
                description="start ride from BIXI"
            )
    return G

In [11]:
def merge_bike_bixi_and_subway(G_bike_bixi, G_subway,
                               bike_subway_transfer_radius_m=BIKE_ACCESS_THRESHOLD_M,
                               bixi_subway_transfer_radius_m=ACCESS_THRESHOLD_M,
                               walking_speed_m_s=WALKING_SPEED_M_S,
                               transfer_penalty_s=STATION_WALK_TRANSFER_PENALTY_S):
    G_combined = nx.MultiDiGraph()

    for n, d in G_bike_bixi.nodes(data=True):
        G_combined.add_node(n, **d)
        
    for n, d in G_subway.nodes(data=True):
        if G_combined.has_node(n):
            existing = G_combined.nodes[n]
            existing.update(d)
            existing["type"] = "station"
        else:
            node_data = d.copy()
            node_data["type"] = "station"
            G_combined.add_node(n, **node_data)

    for u, v, key, data in G_bike_bixi.edges(keys=True, data=True):
        G_combined.add_edge(u, v, **data)

    for u, v, data in G_subway.edges(data=True):
        lines = list(data.get("lines", []))
        weight = data.get("weight")
        dist = data.get("distance_m")
        G_combined.add_edge(u, v, mode="subway", weight=weight, lines=lines, distance_m=dist)
        G_combined.add_edge(v, u, mode="subway", weight=weight, lines=lines, distance_m=dist)

    
    bike_nodes = []
    bike_coords = []
    for n, d in G_bike_bixi.nodes(data=True):
        if d.get("mode") == "bike" or d.get("type") == "bike_node":
            if d.get("lon") is None or d.get("lat") is None:
                continue
            bike_nodes.append(n)
            bike_coords.append(project(d["lon"], d["lat"]))
    bike_coords = np.array(bike_coords) if bike_coords else np.zeros((0, 2))
    bike_tree = cKDTree(bike_coords) if len(bike_coords) > 0 else None

    bixi_nodes = []
    bixi_coords = []
    for n, d in G_bike_bixi.nodes(data=True):
        if str(n).startswith(BIXI_PREFIX):
            if d.get("lon") is None or d.get("lat") is None:
                continue
            bixi_nodes.append(n)
            bixi_coords.append(project(d["lon"], d["lat"]))
    bixi_coords = np.array(bixi_coords) if bixi_coords else np.zeros((0, 2))
    bixi_tree = cKDTree(bixi_coords) if len(bixi_coords) > 0 else None

    subway_nodes = []
    subway_coords = []
    for n, d in G_subway.nodes(data=True):
        if d.get("lon") is None or d.get("lat") is None:
            continue
        subway_nodes.append(n)
        subway_coords.append(project(d["lon"], d["lat"]))
    subway_coords = np.array(subway_coords)
    subway_tree = cKDTree(subway_coords)

    if bike_tree is not None:
        for i, station in enumerate(subway_nodes):
            sx, sy = subway_coords[i]
            idxs = bike_tree.query_ball_point((sx, sy), r=bike_subway_transfer_radius_m)
            if not idxs:
                _, idx = bike_tree.query((sx, sy))
                idxs = [idx]
            for idx in idxs:
                bike_node = bike_nodes[idx]
                bx, by = bike_coords[idx]
                dist = math.hypot(sx - bx, sy - by)
                walk_time = dist / walking_speed_m_s + transfer_penalty_s
                # bidirectional
                G_combined.add_edge(station, bike_node, mode="bike_access", weight=walk_time, distance_m=dist, description="subway<->bike transfer")
                G_combined.add_edge(bike_node, station, mode="bike_access", weight=walk_time, distance_m=dist, description="bike<->subway transfer")

    if bixi_tree is not None:
        for i, station in enumerate(subway_nodes):
            sx, sy = subway_coords[i]
            idxs = bixi_tree.query_ball_point((sx, sy), r=bixi_subway_transfer_radius_m)
            if not idxs:
                _, idx = bixi_tree.query((sx, sy))
                idxs = [idx]
            for idx in idxs:
                bixi_node = bixi_nodes[idx]
                bx, by = bixi_coords[idx]
                dist = math.hypot(sx - bx, sy - by)
                walk_time = dist / walking_speed_m_s + transfer_penalty_s
                G_combined.add_edge(station, bixi_node, mode="walk_access", weight=walk_time, distance_m=dist, description="subway<->bixi walking transfer")
                G_combined.add_edge(bixi_node, station, mode="walk_access", weight=walk_time, distance_m=dist, description="bixi<->subway walking transfer")

    return G_combined

In [12]:
def save_graph_with_metadata(G, out_path, title, components):
    data = json_graph.node_link_data(G)
    
    def default(o):
        if isinstance(o, set):
            return list(o)
        elif hasattr(o, '__dict__'):
            return str(o) 
        elif isinstance(o, (np.integer, np.floating)):
            return o.item() 
        else:
            return str(o)
    
 
    metadata = {
        "generated_at": datetime.datetime.now().isoformat(),
        "title": title,
        "components": components,
        "node_count": G.number_of_nodes(),
        "edge_count": G.number_of_edges(),
    }
    output = {
        "metadata": metadata,
        "graph": data
    }
    
    dir_path = os.path.dirname(out_path)
    if dir_path:
        os.makedirs(dir_path, exist_ok=True)
    
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(output, f, indent=2, ensure_ascii=False, default=default)
    print(f"Saved {title} to {out_path}")

In [17]:
print("=== MULTIMODAL GRAPHS WITH SUBWAY INTEGRATION ===")


print("\nCreating car + subway multimodal graph...")
G_car_gtfs = merge_car_and_subway(G_car, G_subway)
save_graph_with_metadata(G_car_gtfs, "car_with_gtfs.json", "Car + GTFS (Park & Ride)", ["car", "gtfs_subway"])

print("Creating walking + subway multimodal graph...")
G_walk_gtfs = merge_walk_and_subway(G_walk, G_subway)
save_graph_with_metadata(G_walk_gtfs, "walk_with_gtfs.json", "Walk + GTFS", ["walk", "gtfs_subway"])

print("Creating biking + subway multimodal graph...")
G_bike_gtfs = merge_bike_and_subway(G_bike, G_subway)
save_graph_with_metadata(G_bike_gtfs, "bike_with_gtfs.json", "Bike + GTFS", ["bike", "gtfs_subway"])

print("Creating bike + BIXI + subway multimodal graph...")
G_bike_bixi_gtfs = merge_bike_bixi_and_subway(G_bike_bixi, G_subway)
save_graph_with_metadata(G_bike_bixi_gtfs, "bike_bixi_with_gtfs.json", "Bike + BIXI + GTFS", ["bike", "bixi", "gtfs_subway"])

print("\n🎉 ALL TRANSPORT GRAPHS GENERATED SUCCESSFULLY!")
print("\n📊 Generated Graphs Summary:")
print("--- STANDALONE NETWORKS ---")
print("- car_graph.json: Pure driving network")
print("- walk_graph.json: Pure walking/pedestrian network") 
print("- bike_graph.json: Pure cycling network")
print("- gtfs_graph.json: Pure subway/metro network")
print("- bike_bixi_graph.json: Cycling + BIXI bike sharing")
print("\n--- MULTIMODAL NETWORKS ---")
print("- car_with_gtfs.json: Driving + park & ride to subway")
print("- walk_with_gtfs.json: Walking + subway access")
print("- bike_with_gtfs.json: Biking + subway access") 
print("- bike_bixi_with_gtfs.json: Full multimodal biking (bike + BIXI + subway)")

print(f"\n📈 Total: {5 + 4} = 9 comprehensive transport graphs for Montreal!")

=== MULTIMODAL GRAPHS WITH SUBWAY INTEGRATION ===

Creating car + subway multimodal graph...
Saved Car + GTFS (Park & Ride) to car_with_gtfs.json
Creating walking + subway multimodal graph...
Saved Walk + GTFS to walk_with_gtfs.json
Creating biking + subway multimodal graph...
Saved Bike + GTFS to bike_with_gtfs.json
Creating bike + BIXI + subway multimodal graph...
Saved Bike + BIXI + GTFS to bike_bixi_with_gtfs.json

🎉 ALL TRANSPORT GRAPHS GENERATED SUCCESSFULLY!

📊 Generated Graphs Summary:
--- STANDALONE NETWORKS ---
- car_graph.json: Pure driving network
- walk_graph.json: Pure walking/pedestrian network
- bike_graph.json: Pure cycling network
- gtfs_graph.json: Pure subway/metro network
- bike_bixi_graph.json: Cycling + BIXI bike sharing

--- MULTIMODAL NETWORKS ---
- car_with_gtfs.json: Driving + park & ride to subway
- walk_with_gtfs.json: Walking + subway access
- bike_with_gtfs.json: Biking + subway access
- bike_bixi_with_gtfs.json: Full multimodal biking (bike + BIXI + subw

In [18]:
# Convert JSON graphs to Python pickle format for potential Go integration
import pickle
import gzip

def save_graph_as_pickle(G, out_path, title):
    """Save graph as compressed pickle for faster loading"""
    with gzip.open(out_path, 'wb') as f:
        pickle.dump({
            'graph': G,
            'title': title,
            'nodes': G.number_of_nodes(),
            'edges': G.number_of_edges(),
            'generated_at': datetime.datetime.now().isoformat()
        }, f)
    print(f"Saved {title} as pickle to {out_path}")

print("\n=== GENERATING COMPRESSED PICKLE VERSIONS ===")
print("(These load much faster than JSON for large graphs)")

# Convert main standalone graphs to pickle format
save_graph_as_pickle(G_car, "car_graph.pkl.gz", "Car Network")
save_graph_as_pickle(G_walk, "walk_graph.pkl.gz", "Walking Network") 
save_graph_as_pickle(G_bike, "bike_graph.pkl.gz", "Bike Network")
save_graph_as_pickle(G_subway, "gtfs_graph.pkl.gz", "GTFS Subway Network")
save_graph_as_pickle(G_bike_bixi, "bike_bixi_graph.pkl.gz", "Bike + BIXI Network")

print("\n📦 Pickle files generated for faster loading in Python!")
print("Note: For Go server integration, the JSON files are recommended.")


=== GENERATING COMPRESSED PICKLE VERSIONS ===
(These load much faster than JSON for large graphs)
Saved Car Network as pickle to car_graph.pkl.gz
Saved Walking Network as pickle to walk_graph.pkl.gz
Saved Bike Network as pickle to bike_graph.pkl.gz
Saved GTFS Subway Network as pickle to gtfs_graph.pkl.gz
Saved Bike + BIXI Network as pickle to bike_bixi_graph.pkl.gz

📦 Pickle files generated for faster loading in Python!
Note: For Go server integration, the JSON files are recommended.


In [16]:
# Generate standalone transport graphs and multimodal graphs
print("=== GENERATING STANDALONE TRANSPORT GRAPHS ===")

# 1. Standalone Car Graph
print("Creating standalone car graph...")
G_car = create_car_graph()
save_graph_with_metadata(G_car, "car_graph.json", "Car Network", ["car"])

# 2. Standalone Walking Graph  
print("Creating standalone walking graph...")
G_walk = create_walk_graph()
save_graph_with_metadata(G_walk, "walk_graph.json", "Walking Network", ["walk"])

# 3. Standalone Bike Graph
print("Creating standalone bike graph...")
G_bike = create_bike_graph()
save_graph_with_metadata(G_bike, "bike_graph.json", "Bike Network", ["bike"])

# 4. Standalone GTFS/Subway Graph
print("Downloading and processing GTFS data...")
routes, trips, stop_times, stops = download_and_extract_gtfs()
G_subway = build_subway_graph_from_gtfs(routes, trips, stop_times, stops)
save_graph_with_metadata(G_subway, "gtfs_graph.json", "GTFS Subway Network", ["gtfs_subway"])
print(f"Built subway graph: {G_subway.number_of_nodes()} nodes, {G_subway.number_of_edges()} edges")

# 5. Standalone Bike + BIXI Graph
print("Fetching BIXI station data...")
stations_df, status_df = fetch_bixi_gbfs()
G_bike_bixi = integrate_bixi_as_endpoints(G_bike, stations_df, status_df)
save_graph_with_metadata(G_bike_bixi, "bike_bixi_graph.json", "Bike + BIXI Network", ["bike", "bixi"])

print("\n=== GENERATING MULTIMODAL TRANSPORT GRAPHS WITH SUBWAY INTEGRATION ===")

# Now generate all the multimodal combinations

=== GENERATING STANDALONE TRANSPORT GRAPHS ===
Creating standalone car graph...
Saved Car Network to car_graph.json
Creating standalone walking graph...
Saved Walking Network to walk_graph.json
Creating standalone bike graph...
Saved Bike Network to bike_graph.json
Saved GTFS Subway Network to gtfs_graph.json
Built subway graph: 72 nodes, 69 edges
Fetching BIXI station data...
Saved Bike + BIXI Network to bike_bixi_graph.json

=== GENERATING MULTIMODAL TRANSPORT GRAPHS WITH SUBWAY INTEGRATION ===


In [ ]:
# Test: Generate just the subway graph first to verify everything works
print("Testing: Downloading and processing GTFS data...")
routes, trips, stop_times, stops = download_and_extract_gtfs()
G_subway = build_subway_graph_from_gtfs(routes, trips, stop_times, stops)
print(f"✅ Built subway graph: {G_subway.number_of_nodes()} nodes, {G_subway.number_of_edges()} edges")

# Test car graph creation
print("\nTesting car graph creation...")
G_car = create_car_graph()
print(f"✅ Built car graph: {G_car.number_of_nodes()} nodes, {G_car.number_of_edges()} edges")

print("\n🎉 All basic functions are working! Ready to generate full graphs.")

Testing: Downloading and processing GTFS data...
✅ Built subway graph: 72 nodes, 69 edges

Testing car graph creation...
✅ Built subway graph: 72 nodes, 69 edges

Testing car graph creation...
✅ Built car graph: 19515 nodes, 48843 edges

🎉 All basic functions are working! Ready to generate full graphs.
✅ Built car graph: 19515 nodes, 48843 edges

🎉 All basic functions are working! Ready to generate full graphs.


In [ ]:
# Test the save function
print("Testing save function...")
save_graph_with_metadata(G_subway, "test_subway.json", "Test Subway", ["gtfs_subway"])
print("✅ Save function works!")

Testing save function...
Saved Test Subway to test_subway.json
✅ Save function works!
